# Prerequisities

In [0]:
df = spark.read.csv(path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/employee.csv",
                    sep="|",
                    inferSchema=True,
                    header=True,
                    quote="'")
display(df)        

# Select Operations

In [0]:
df.select("id","name").display()

In [0]:
from pyspark.sql.functions import col
df.select(col("id").alias("user_id"),col("name")).display()

In [0]:
df.filter(
    col("desig")=='Developer'
).display()

In [0]:
df.filter(
    (col("desig")=='Developer') 
    |
    (col("desig")=='Team Lead') 
).select("company").display()

In [0]:
df.filter(
    (col("desig")=='Developer') 
    &
    (col("gen")=='M') 
).display()

In [0]:
df.filter(
    (col("exp").between(1,6))
        &
    (col("gen")=='M') 
).select("name").display()

# df.filter(
#     col("exp") >= 1 & col("exp") <= 6
# ).display()

In [0]:
df.filter((col("exp").between(1, 6))
          ).filter(
              (col("gen") == "M")
          ).select("name").display()

# Grouping, agg, sorting

In [0]:
df.groupBy(col("gen")).count().display()

In [0]:
df.groupBy(col("gen")).count().sort(col("count"), ascending = False).display()

df.groupBy(col("gen")).count().sort(col("count"), ascending = False).display()

In [0]:
df.groupBy(col("gen")).count().sort(col("count").desc()).select(
    col("gen"), col("count").alias("total_count")
).display()

In [0]:
from pyspark.sql.functions import avg, sum, min, max
df.groupBy("desig").agg(min("exp").alias("min_exp"),
                        max("exp").alias("max_exp"),
                        avg("exp").alias("avg_exp")).display()

# Derive new columns

In [0]:
from pyspark.sql.functions import lit
df.withColumn("is_employeed",lit("Yes")).display()

In [0]:
from pyspark.sql.functions import when

df.withColumn(
    "category",
    when(col("exp") >= 10, "Senior")
    .when(col("exp") >= 5, "Mid Level")
    .when(col("exp") >= 1, "Junior")
    .otherwise("Invalid Experience"),
).display()

# View

In [0]:
df.createOrReplaceTempView("employee_vw")
spark.sql("select * from employee_vw").display()
# spark.sql("select count(*) from employee").display()

In [0]:
%sql

select * from employee_vw

In [0]:
%sql
SELECT *,
CASE
    WHEN exp >=10 THEN 'Senior'
    WHEN  exp >=5 THEN 'Mid level'
     WHEN  exp >=0 THEN 'Junior'
    ELSE 'Invalid Experience'
END AS category
FROm employee_vw